<font size="+3"><center>
    CCATpHive Documentation
</center></font>

<center><b>James Burgoyne</b></center>
<center>jburgoyne@phas.ubc.ca</center>
<center>May 2022</center>

# Introduction

CCATpHive is python software to mediate communication between the control server and the parallel operating RFSoC boards which control readout operations for the CCAT Prime telescope. Development was begun in January 2022. The software's main goals are: (1) to provide interface hooks (and a terminal interface) to board functionality; (2) to integrate with a Redis server to communicate over a network between the control server and the boards; (3) to provide control of the main readout functionality scripts. 

This document is organized as follows:

## Current Status (June 2022)

As of June 2022, a basic working version which accomplishes the 3 goals is complete. Future work will focus on bug-fixes and improvements.

# Environment

## Python

For development and testing, python 3.7 was used.

# File Structure

- **_cfg_board.py**: Board configuration options.
- **queen.py**: Runs on the command server to publish commands to remote boards, and to listen for messages from the boards. Should be running at all times in listen mode to pick up board messages.
- **queen_tui.py**: A terminal interface to queen.py.
- **drone.py**: Runs on each of the boards (4 instances) and listens for commands. Upon receiving a command it asks alcove.py to execute it and publishes any necessary messages, e.g. the return statement. Should be running at all times to hear commands.
- **alcove.py**: Mediator to the board functionality functions (commands).
- **alcove_tui.py**: A terminal interface to alcove.py.
- **/alcove_commands**: Modules containing the functions to perform board tasks.
    - **board_utilities.py**: Basic board utility tools.
    - **test_functions.py**: Testing functions.
- **/queen_commands**: Modules containing the functions to perform server tasks.
    - **test_functions**: Testing functions.

# Redis Channel Structure

# Usage

Basic command flow is as follows: 
1. Redis server startup (on control server).
1. Listening Queen startup (on control server).
1. Drones startup (on boards - multiple drones per board, multiple boards).
1. Commands sent to Queen (e.g. via queen_tui.py) are communicated to redis in appropriate channels. Drones are subscribed to redis and execute commands they receive on their channels.

## Redis Server
A Redis server must be running for the control server and boards to communicate. The configration options (e.g. host, port, etc.) for the Redis server are in ```_cfg_board.py``` for the boards, and in ```queen.py``` for the control server. The Redis server will be hosted on the control server.

Start Redis server (ampersand is optional - allows sending shutdown command in same terminal):
```bash
redis-server &
```

Stop Redis server:
```bash
redis-cli shutdown
```

<figure>
  <img src="redis_startup.png" alt="Redis startup in the terminal."/>
  <figcaption>Redis server startup from a terminal. Note that if the ampersand wasn't addended, another terminal would be needed to shutdown the server.</figcaption>
</figure>

## Drone

Each board runs 4 drones (drone.py) in parallel (separated into 4 subdirectories), each operating a separate 512 MHz bandwidth microwave network (divided into 1000 channels). Each drone operates in a 'listening' mode in order to implement commands received from the queen over redis. The drones must be running at all times, and should be started at board startup, and up-status monitored via a daemon.

To start a single instance of drone.py (in listening mode):
```bash
python drone.py
```

## Queen (TUI)

Queen (queen.py) can potentially be interacted with from different user-interfaces, but at time-of-writing the only interface is a terminal-user-interface (queen_tui.py).

### Help

To get **help** with queen_tui:
```bash
python queen_tui.py -h
```

### Commands

All commands have an associated command number which is given as an argument.

To see a list of possible **commands**:
```bash
python queen_tui.py --commands
```

The **recipient** of a command sent to queen_tui must be explicitly declared as an argument. 
- ```--queen```: Command for queen itself.
- ```--all```: Command for all boards.
- ```--bid [bid]```: Command for board with board-identifier (bid) of [bid], where [bid] is an integer. Board identifiers are set in *_cfg_board.py* within each of the 4 drone subdirectories.

The command number is declared with the argument ```-c [cid]``` where [cid] is the integer command number.

### Listen Mode

An instance of queen (queen.py) should be running in 'listen' mode at all times to process board messages. 
To start a single instance of queen.py in **listen mode**:
```bash
python queen_tui.py --queen -c 21
```